# City Benches in Manhattan

## What is the shortest distance between City Benches in Manhattan?

### Noah Nasarek, Manuel Dedio and Daria Zakharova

In [26]:
#Necessary packages
import pandas as pd
import osmnx as ox
import networkx as nx
import pickle
import geopandas 
import matplotlib.pyplot as plt


## Data Import

We import our raw data from https://www1.nyc.gov/html/dot/downloads/pdf/citybench-locations.pdf and convert it into an .xlsx using https://smallpdf.com/de/pdf-in-excel 

In [27]:
df = pd.read_excel('citybenchlocationsv03.xlsx')
df.head(20)

,Bench\nCount,Address,On Street,Cross Streets,Borough,Bench\nType,Installation\nDate
0,1,540 West 145th Street,West 145th Street,Broadway and Amsterdam Avenue,Manhattan,backless,2012-05-17
1,2,110 East 149th Street,West 149th Street,Gerard Avenue River Avenue,Bronx,backless,2012-05-16
2,3,125 East 149th Street,East 149th Street,Gerard Avenue & Walton Avenue,Bronx,backless,2012-05-16
3,4,360 East 149th Street,East 149th Street,Courtlandt and Melrose Avenues,Bronx,backless,2012-05-16
4,5,414 & 416 East 149th Street,East 149th Street,Bergen and Third Avenues,Bronx,backless,2012-05-16
5,6,700 Southern Boulevard,Southern Boulevard,Leggett Avenue and East 156th Street,Bronx,backless,2012-05-16
6,7,302 Broadway,Broadway,Marcy Avenue and Rodney Street,Brooklyn,backless,2012-04-14
7,8,825 Broadway & 829 Broadway,Broadway,Park Street and Ellery Street,Brooklyn,backless,2012-04-14
8,9,893 Broadway,Broadway,Arion Place and Belvedere Street,Brooklyn,backless,2012-04-14
9,10,57 Malcolm X Boulevard,Malcolm X Boulevard,Lafayette Avenue & Van Buren Street,Brooklyn,backless,2013-12-13


## Data Wrangling

The data after several rounds of Data Wrangling


In [33]:
addressm = pd.read_pickle("addressm.pkl")
addressm.head(20)

,Address,Borough,Geocode,Node
0,"540 West 145th Street, New York City",Manhattan,"(40.8260052, -73.949854)",561035361
56,"145 & 147 Madison Ave, New York City",Manhattan,"(40.74619633333333, -73.983977)",3252586222
57,"165 Madison Avenue, New York City",Manhattan,"(40.746812750000004, -73.98327882871746)",42445903
58,"191 Madison Avenue, New York City",Manhattan,"(40.74802628571428, -73.98264385714285)",4960503208
59,"199 Madison Avenue, New York City",Manhattan,"(40.7482878, -73.9824103)",4960503208
60,"227 Madison Avenue, New York City",Manhattan,"(40.7598067, -73.9741741)",3977886242
88,"175 East 96th Street, New York City",Manhattan,"(40.785616, -73.9498645)",7325349531
91,"201 West 106th Street, New York City",Manhattan,"(40.8006923, -73.9659254)",4205830395
94,"1020 Third Avenue, New York City",Manhattan,"(40.812824, -73.923829)",7477281986
99,"209 West 96th Street, New York City",Manhattan,"(40.7946572, -73.97102558044648)",4205805396


## Distance Calculation

We calculate the distance for every bench pair

In [34]:
with open("distance.txt", "rb") as fp:
    d = pickle.load(fp)
print(f"The first 100 entries of the list: \n \n {d[0:50]}", d[:50])

The first 100 entries of the list: 
 
 [9994.472000000002, 9925.326000000001, 9773.585000000001, 9773.585000000001, 8551.925999999994, 5455.073999999999, 3323.094000000002, 2902.405000000001, 4030.3600000000024, 14850.394000000015, 14850.394000000015, 12242.41100000001, 4066.8970000000027, 4066.8970000000027, 3402.928, 5466.083, 6296.718, 6296.718, 2258.3079999999995, 2227.524, 3327.741, 12235.783000000001, 12235.783000000001, 12328.36, 12328.36, 12721.303000000007, 12721.303000000007, 12305.416000000005, 12242.918000000009, 12242.918000000009, 12223.967000000004, 12223.967000000004, 14771.25600000001, 14750.004000000008, 14672.788000000002, 14559.203000000005, 14460.520000000013, 14479.474000000011, 14580.403000000013, 14580.403000000013, 14477.889000000012, 14384.918000000012, 14397.845000000014, 14395.584000000013, 7352.452000000001, 6404.790999999997, 13947.251000000002, 10119.563999999997, 3856.1220000000003, 10625.045000000004] [9994.472000000002, 9925.326000000001, 9773.58500000

## Shortest Distance

In [35]:
print(f"The shortest distance is {sorted(list(set(d)))[1]}m.")

The shortest distance is 1m.


In [38]:
shortest_dist = []
i = 0
while i <= 50: 
    i += 1
    shortest_dist.append(sorted(list(set(d)))[i])

In [39]:
print("The 50 shortest distances are: \n", shortest_dist)

The 50 shortest distances are: 
 [1, 8.436, 9.237, 9.383, 9.439, 9.729, 15.943, 18.302, 18.347, 19.506999999999998, 20.442999999999998, 20.776, 22.659, 23.884999999999998, 24.293, 25.003, 25.489, 25.771, 31.702, 32.042, 38.221, 39.783, 40.881, 41.208, 41.653, 42.257000000000005, 46.785, 47.004, 48.293, 48.509, 51.894, 52.608000000000004, 53.57599999999999, 53.576, 54.632000000000005, 54.661, 56.51899999999999, 56.519, 61.297000000000004, 61.532, 62.348, 68.304, 71.233, 71.904, 71.90400000000001, 72.134, 73.7, 75.727, 76.523, 77.93499999999999, 78.454]


# Thank You!